In [ ]:
#note check resample?

import cv2 as cv
# import cv as cv1
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
from numpy import random
from dataclasses import dataclass
from math import exp

PINK = (179, 102, 255)
SET_ROI = False
corner_points = []

# @dataclass
class face:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.w = 0
        self.h = 0
        self.dx = 0 
        self.dy = 0
        self.confidence = 0
        self.weight = 0
        self.rHist = 0
        self.gHist = 0
        self.bHist = 0
    def get_rect(self):
        return(self.x, self.y, self.w, self.h)
    def set_rect(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        
class rect:
    def __init__(self, x = 0, y  = 0, w = 0, h = 0):
        self.x = x, self.y = y, self.w = w, self.h = h
        return
def CreateImage(width, height, rgb_color=(0, 0, 0)):
    image = np.zeros((height, width, len(rgb_color)), np.uint8)

    # Since OpenCV uses BGR, convert the color first
    color = tuple(reversed(rgb_color))
    # Fill image with color
    image[:] = color
    return image

def getHist(im, x, y, w, h):
#     cv.SetImageROI(im, (x,y,w,h))
#     try:
#         ROI = im[x:x+w, y:y+h]
#     except:
#         print('\nx:{}'.format(x) + '\ny:{}'.format(y) + '\nw:{}'.format(w) + '\nh:{}'.format(h))
#         quit()
    ROI = im[x:x+w, y:y+h]    
    numBins = 64
    rangez = [0.0, 255.0]
    ####
#     gray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
#     if im == None:
#         print('no image')
    h = cv.calcHist([ROI], [0], mask = None, histSize = [64], ranges = rangez)
#     h = cv.calcHist(im, 1, numBins, CV_HIST_ARRAY,ranges = range)

    ####
#     cv.ResetImageROI(im)
    return h

def getSamples(f, predX, predY, predW, predH, numSamples, searchSTD, wLimit, hLimit):
    samples = []
    
    #generate random samples
    for randGenIter in range(numSamples):
        #generate two random uniformly dist numbers
        n1 = random.normal(0,1)
        n2 = random.normal(0,1)
        n3 = random.normal(0,1)
        
        #make std dev 1/3 of face dim and mean at the predicted position
        n1 *= f.w * searchSTD
        n1 += predX
        n2 *= f.h * searchSTD
        n2 += predY
        
        n3 = 1
        
        scale = n3
        newWidth = predW * scale
        newHeight = predH * scale
        
        if (n1 > 0) and (n1 < wLimit - newWidth) and (n2 < hLimit - newHeight):#if random position is on the image
            #declare a face at the location
            newFace = face()
            #newRect = rect()
#             newFace.rect = rect(n1, n2, newWidth, newHeight)
            newFace.x = int(n1)
            newFace.y = int(n2)
            newFace.w = int(newWidth)
            newFace.h = int(newHeight)
            global imR 
            global imG 
            global imB
            newFace.rHist = getHist(imR, newFace.x, newFace.y, newFace.w, newFace.h)
            newFace.gHist = getHist(imG, newFace.x, newFace.y, newFace.w, newFace.h)
#             newFace.gHist = getHist(imG, newFace.y, newFace.w, newFace.h)
            newFace.bHist = getHist(imB, newFace.x, newFace.y, newFace.w, newFace.h)
#             newFace.bHist = getHist(imB, newFace.y, newFace.w, newFace.h)
    
            newFace.dx = 0
            newFace.dy = 0
            
            #calc liklihood/weight
            rCorr = cv.compareHist(newFace.rHist, f.rHist, cv.HISTCMP_CORREL)
            gCorr = cv.compareHist(newFace.gHist, f.gHist, cv.HISTCMP_CORREL)
            bCorr = cv.compareHist(newFace.bHist, f.bHist, cv.HISTCMP_CORREL)
            
            likelihood = (rCorr*0.4 + gCorr*0.3 + bCorr*0.3)
            newFace.weight = exp(-16.0 * (1-likelihood))
            samples.append(newFace)
    return samples

def resample(samples, f, wLimit, hLimit):
    totalWeight = 0.0
    for sampleIter in range(len(samples)):
        totalWeight += samples[sampleIter].weight
        resamples = []
        allResamples = []
        numSamplesToDraw = 0
        for sampleIter in range(len(samples)):
            resamples.clear()
            numSamplesToDraw = int(((samples[sampleIter].weight / totalWeight) * len(samples)) + 0.5)
            print('{} samples drawn'.format(numSamplesToDraw))
            #predicted position
            predX = samples[sampleIter].x
            predY = samples[sampleIter].y
            
            resamples = getSamples(f, predX, predY, 
                                   samples[sampleIter].w, samples[sampleIter].h,
                                   numSamplesToDraw, 0.1, wLimit, hLimit)
            #add resamples to the vector of all resamples
            for resampleIter in range(len(resamples)):
                allResamples.append(resamples[resampleIter])
    return allResamples

def drawFaces():
    #copy the image and draw the faces
    global imFace 
    global im
    imFace = im.copy()
    pt1 = [0,0]
    pt2 = [0,0]
    rectColor = 0
    #draw the faces
    for faceIter in range(len(allFaces)):
        pt1[0] = allFaces[faceIter].x
        pt2[0] = pt1[0] + allFaces[faceIter].w
        pt1[1] = allFaces[faceIter].y
        pt2[1] = pt1[1] + allFaces[faceIter].h
        
        pt1 = tuple(pt1)
        pt2 = tuple(pt2)
        rectColor = (0,0,0,0)
        cv.rectangle(imFace, pt1, pt2, rectColor, 3, 8, 0)
        rectColor = (0,255,0,0)  
        cv.rectangle(imFace, pt1, pt2, rectColor, 1, 8, 0)
        
    #cv.imshow('Faces', imFace)
    plt.imshow(cv.cvtColor(imFace,cv.COLOR_BGR2RGB))
    plt.show()

def run():
    print('\nWait...')
    cv.waitKey(3000)
    print('\nGo!')
    global im
    global imFace
#     load haar classifier
    cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
#     storage = cv.CreateMemStorage(0)
    cap = cv.VideoCapture(0)
    ret, im = cap.read()
    #writer = cv.CreateVideoWriter
    imFace = im.copy() 
    
    #allocate some images used to extract a skin likelihood map
    imHSV = CreateImage(im.shape[1], im.shape[0], (0,0,0))
    skin = CreateImage(im.shape[1], im.shape[0], (0,))
    global imR
    global imG
    global imB
    imR = CreateImage(im.shape[1], im.shape[0], (0,))
    imG = CreateImage(im.shape[1], im.shape[0], (0,))
    imB = CreateImage(im.shape[1], im.shape[0], (0,))
    
    sampling = CreateImage(im.shape[1], im.shape[0], (0,))
    
    frame = 0
    keyPressed = -1
    writeActive = True
    i = 1
#     while True:
    while (i!=0) and ((keyPressed ==-1) or (keyPressed == 'f')):
        cv.imshow('Given', im)
        imHSV = cv.cvtColor(im, cv.COLOR_BGR2HSV)
#         cv.CvtColor(im, imHSV, cv.CV_BGR2HSV)
        pixRGB = 0
        pixHSV = 0
        cb = 0.0
        cr = 0.0
        
        #separate into channels
#         cv.SetImageCOI(im,1)
        imB = im[:,:,0].copy()
#         cv.SetImageCOI(im,2)
        imG = im[:,:,1].copy()
#         cv.SetImageCOI(im,3)
        imR = im[:,:,2].copy()
#         cv.SetImageCOI(im,0)
        
        if keyPressed == 'w':
            writeActive = !writeActive
        if (frame == 0) or (len(allFaces) == 0) or (keyPressed == 'f'): #detect faces
#             cv.ClearMemStorage(storage) #clear earlier memory
#             faces = cv.HaarDetectObjects(im, cascade, storage, 1.1, 2,
#                                          cv.CV_HAAR_DO_CANNY_PRUNING, (40,40))
            faces = cascade.detectMultiScale(im, 1.3, 5)
            for currentFace in range(len(faces)):
                #get rectangle bounding first face
                x, y, w, h = faces[currentFace]
                #############################
                if (type(x) == float):
                    print(x)
                    return
                #############################
                newFace = face()
                newFace.set_rect(x,y,w,h)
                newFace.confidence = 2
                newFace.dx = 0
                newFace.dy = 0
                
                #find total amount of skin color pixels in the face
                skinSum = 0
                for yy in range(newFace.y, newFace.y + newFace.h):
                    for xx in range(newFace.x, newFace.x + newFace.w):
                        #pixRGB = cv.Get2D(im,y,x)
                        pixRGB = im[y,x,:]
                        #pixHSV = cv.Get2D(imHSV,y,x)
                        pixHSV = imHSV[y,x,:]
                        cb = 0.148*pixRGB[2] - 0.291*pixRGB[1] + 0.439*pixRGB[0] + 128
                        cr = 0.439*pixRGB[2] - 0.368 * pixRGB[1] - 0.071*pixRGB[0] + 128
                        if ((pixHSV[0] > 245) or (pixHSV[0] < 25.5)) and (140<=cr) and (cr<=165) and (140<=cb) and (cb<=195):
                            skinSum +=1
                if skinSum / (newFace.w * newFace.h) < 0.3: #less than 30% skin
                    break
                #check to see if face is rough match to existing
                matchesExisting = False
                for faceIter in range(len(allFaces)):
                    #find width and height of region of overlap
                    overlapWidth = 0
                    overlapHeigh = 0
                    if newFace.x < allFaces[faceIter].x:
                        overlapWidth = min(newFace.x + newFace.w - allFaces[faceIter].x,
                                          allFaces[faceIter].width)
                    else:
                        overlapWidth = min(allFaces[faceIter].x + allFaces[faceIter].w - newFace.x,
                                          newFace.w)
                    if newFace.y < allFaces[faceIter].y:
                        overlapHeight = min(newFace.y + newFace.h - allFaces[faceIter].y,
                                           allFaces[faceIter].h)
                    else:
                        overlapHeight = min(allFaces[faceIter].y + allFaces[faceIter].h - newFace.y,
                                           newFace.h)

                    #if region of overlap is greater than 60% of larger rect, the faces are the same
                    if ((overlapWidth * overlapHeight) / max(newFace.w * newFace.h, allFaces[faceIter].w* allfaces[faceIter].h)) > 0.6:
                        matchesExisting = True
                        allFaces[faceIter].confidence = min(4, allFaces[faceIter].confidence + 2)
                        #set to newface
                        allFaces[faceIter].x = newFace.x
                        allFaces[faceIter].y = newFace.y
                        allFaces[faceIter].w = newFace.w
                        allFaces[faceIter].h = newFace.h
                        allFaces[faceIter].dx = newFace.dx
                        allFaces[faceIter].dy = newFace.dy
                        allFaces[faceIter].rHist = getHist(imR,
                                                           (newFace.x,
                                                           newFace.y,
                                                           newFace.w,
                                                           newFace.h,))
                        allFaces[faceIter].gHist = getHist(imG,
                                                           (newFace.x,
                                                           newFace.y,
                                                           newFace.w,
                                                           newFace.h,))
                        allFaces[faceIter].bHist = getHist(imB,
                                                           (newFace.x,
                                                           newFace.y,
                                                           newFace.w,
                                                           newFace.h,))
                        break
                if matchesExisting == 0: #if new face, add to our vector
                    newFace.rHist = getHist(imR, newFace.x, newFace.y, newFace.w, newFace.h)
                    newFace.gHist = getHist(imG, newFace.x, newFace.y, newFace.w, newFace.h)
                    newFace.bHist = getHist(imB, newFace.x, newFace.y, newFace.w, newFace.h)
                    allFaces.append(newFace)
            for faceIter in range(len(allFaces)): #subtract 1 from confidence of all faces
                allFaces[faceIter].confidence -= 1
                if allFaces[faceIter].confidence < 1: #if confidence is gone, remove it
                    del allFaces[faceIter]
                    faceIter -= 1
        if len(allFaces) > 0: #track faces
#             cv.cvSet(sampling, 0)
            sampling = CreateImage(im.shape[1], im.shape[0], (0,))
            for faceIter in range(len(allFaces)): #first face only for now
                predX = int(allFaces[faceIter].x + allFaces[faceIter].dx)
                predY = int(allFaces[faceIter].y + allFaces[faceIter].dy)
                samples = getSamples(allFaces[faceIter], predX, predY, allFaces[faceIter].w, allFaces[faceIter].h, 100, 0.2, im.shape[1], im.shape[0])
                #importance resampling
                for resampling in range(3):
                    samples = resample(samples, allFaces[faceIter], im.shape[1], im.shape[0])
                bestIdx = 0
                bestWeight = 0
                
                #generate random samples
                for sampleIter in range(len(samples)):
                    if samples[sampleIter].weight > bestWeight:
                        bestWeight = samples[sampleIter].weight
                        bestIdx = sampleIter
                #move to best sample
                allFaces[faceIter].dx = samples[bestIdx].x - allFaces[faceIter].x
                allFaces[faceIter].dy = samples[bestIdx].y - allFaces[faceIter].y
                allFaces[faceIter].x = samples[bestIdx].x
                allFaces[faceIter].y = samples[bestIdx].y
                allFaces[faceIter].w = samples[bestIdx].w
                allFaces[faceIter].h = samples[bestIdx].h
                #copy hists?
            drawFaces()
        i, im = cap.read()
        frame += 1
        
        keyPressed = cv.waitKey(10)
        print('Iteration!')
    cap.release()
    cv.destroyAllWindows()
                
                
            
        
                    
                                            
                                           
                
                                                              

                        
                

In [ ]:
allFaces = []
imR = 0
imB = 0
imG = 0
im = 0
run()

In [ ]:
cv.IPL_DEPTH_8U

In [ ]:
# face_detect = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     print("Cannot open camera")
#     exit()
# # ret, frame = cap.read()
# while True:
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     # if frame is read correctly ret is True
#     if not ret:
#         print("Can't receive frame (stream end?). Exiting ...")
#         break
#      # Our operations on the frame come here
#     face_data = face_detect.detectMultiScale(frame, 1.3, 5)
    
#     for (x, y, w, h) in face_data:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         roi = frame[y:y+h, x:x+w]
#         # applying a gaussian blur over this new rectangle area
#         roi = cv2.GaussianBlur(roi, (23, 23), 30)
#         # impose this blurred image on original image to get final image
#         frame[y:y+roi.shape[0], x:x+roi.shape[1]] = roi
    
# #     gray = cv2.cvtColor(frame, cv.COLOR_BGR2GRAY)
#      # Display the resulting frame
# #     cv2.imshow('frame', gray)
#     cv2.imshow('frame', frame)
#     if cv2.waitKey(1) == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
type(np.uint8)

In [ ]:
cv.SetImageCOI()

In [ ]:
cv.cv

In [ ]:
cap.release()

In [ ]:
class test:
    def __init__(self):
        self.x = 0, self.y = 0
zz = test()


In [ ]:
cv.imshow('1', imB)

In [ ]:
h = int(5.537382)
h

In [ ]:
z = 4
if z == 4:
    print('z is 4 dawg')
    quit()
print('no quit, z is {}'.format(z))

In [ ]:
import cv2

cap = cv2.VideoCapture(0)
# cap.open('https://videos3.earthcam.com/fecnetwork/9974.flv/chunklist_w1421640637.m3u8')

while (True):
    _, frame = cap.read()
    cv2.imshow("camCapture", frame)
    cv2.waitKey(5000)

In [ ]:
print(frame)